In [1]:
GPU_device_id = str(2)
model_id_save_as = 'learningcurve-dnn-full-final'
architecture_id = 'final-models/learningcurve-dnn-full-final-features'
model_class_id = 'DNN'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_full_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_full_200keV_log10time_10000.npy'
difficulty_setting = 'full'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_dnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [6]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [4]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.softmax
model_features.metrics = [f1]
model_features.learning_rate = model_features.learining_rate

In [ ]:
model = compile_model(
            build_dnn_model,
            model_features)

## Train network

# Scale input data

In [30]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [31]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [32]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        model = compile_model(
            build_dnn_model,
            model_features)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 20000
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 5s 262us/sample - loss: 28.1272 - f1: 0.2160 - val_loss: 17.2856 - val_f1: 0.3405
Epoch 2/2000
20000/20000 [==============================] - 3s 142us/sample - loss: 13.8144 - f1: 0.3687 - val_loss: 10.9356 - val_f1: 0.4707
Epoch 3/2000
20000/20000 [==============================] - 3s 143us/sample - loss: 9.4127 - f1: 0.4446 - val_loss: 7.7528 - val_f1: 0.5480
Epoch 4/2000
20000/20000 [==============================] - 3s 143us/sample - loss: 6.9940 - f1: 0.5022 - val_loss: 6.0426 - val_f1: 0.5669
Epoch 5/2000
20000/20000 [==============================] - 3s 142us/sample - loss: 5.4981 - f1: 0.5321 - val_loss: 4.7148 - val_f1: 0.6187
Epoch 6/2000
20000/20000 [==============================] - 3s 142us/sample - loss: 4.4918 - f1: 0.5624 - val_loss: 3.8962 - val_f1: 0.6337
Epoch 7/2000
20000/20000 [==============================] - 3s 148us/sample - loss: 3.7654 - f1: 0.5907 - v

Epoch 17/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 1.5376 - f1: 0.7295 - val_loss: 1.4242 - val_f1: 0.7610
Epoch 18/2000
20000/20000 [==============================] - 3s 151us/sample - loss: 1.4546 - f1: 0.7417 - val_loss: 1.4822 - val_f1: 0.7398
Epoch 19/2000
20000/20000 [==============================] - 3s 145us/sample - loss: 1.4106 - f1: 0.7398 - val_loss: 1.3645 - val_f1: 0.7633
Epoch 20/2000
20000/20000 [==============================] - 3s 145us/sample - loss: 1.3340 - f1: 0.7562 - val_loss: 1.2793 - val_f1: 0.7714
Epoch 21/2000
20000/20000 [==============================] - 3s 146us/sample - loss: 1.3213 - f1: 0.7488 - val_loss: 1.2583 - val_f1: 0.7637
Epoch 22/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 1.2615 - f1: 0.7602 - val_loss: 1.2054 - val_f1: 0.7775
Epoch 23/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 1.2380 - f1: 0.7601 - val_loss: 1.2498 - val_f1: 0.7498
Epoch 24/2000

Epoch 46/2000
20000/20000 [==============================] - 3s 146us/sample - loss: 0.8796 - f1: 0.8154 - val_loss: 0.9721 - val_f1: 0.7984
Epoch 47/2000
20000/20000 [==============================] - 3s 145us/sample - loss: 0.8939 - f1: 0.8117 - val_loss: 0.8114 - val_f1: 0.8362
Epoch 48/2000
20000/20000 [==============================] - 3s 150us/sample - loss: 0.8705 - f1: 0.8193 - val_loss: 0.9255 - val_f1: 0.8134
Epoch 49/2000
20000/20000 [==============================] - 3s 146us/sample - loss: 0.8593 - f1: 0.8199 - val_loss: 0.7733 - val_f1: 0.8590
Epoch 50/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 0.8650 - f1: 0.8182 - val_loss: 0.8690 - val_f1: 0.8202
Epoch 51/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 0.8615 - f1: 0.8184 - val_loss: 0.8311 - val_f1: 0.8322
Epoch 52/2000
20000/20000 [==============================] - 3s 147us/sample - loss: 0.8617 - f1: 0.8203 - val_loss: 0.8255 - val_f1: 0.8391
Epoch 53/2000

...

Epoch 20/2000
10000/10000 [==============================] - 2s 165us/sample - loss: 2.3820 - f1: 0.6792 - val_loss: 2.2649 - val_f1: 0.7043
Epoch 21/2000
10000/10000 [==============================] - 2s 166us/sample - loss: 2.2748 - f1: 0.6796 - val_loss: 2.2961 - val_f1: 0.6686
Epoch 22/2000
10000/10000 [==============================] - 2s 165us/sample - loss: 2.1568 - f1: 0.6911 - val_loss: 2.1157 - val_f1: 0.7063
Epoch 23/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 2.0400 - f1: 0.7036 - val_loss: 2.0194 - val_f1: 0.7115
Epoch 24/2000
10000/10000 [==============================] - 2s 166us/sample - loss: 1.9998 - f1: 0.7025 - val_loss: 1.8788 - val_f1: 0.7313
Epoch 25/2000
10000/10000 [==============================] - 2s 165us/sample - loss: 1.8964 - f1: 0.7149 - val_loss: 1.8254 - val_f1: 0.7288
Epoch 26/2000
10000/10000 [==============================] - 2s 166us/sample - loss: 1.8679 - f1: 0.7034 - val_loss: 1.9810 - val_f1: 0.6651
Epoch 27/2000

Epoch 47/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.2034 - f1: 0.7739 - val_loss: 1.1568 - val_f1: 0.7842
Epoch 48/2000
10000/10000 [==============================] - 2s 166us/sample - loss: 1.2024 - f1: 0.7698 - val_loss: 1.2346 - val_f1: 0.7604
Epoch 49/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.1779 - f1: 0.7754 - val_loss: 1.1007 - val_f1: 0.8011
Epoch 50/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.1263 - f1: 0.7865 - val_loss: 1.0935 - val_f1: 0.8063
Epoch 51/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.1092 - f1: 0.7906 - val_loss: 1.2835 - val_f1: 0.7459
Epoch 52/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.1340 - f1: 0.7794 - val_loss: 1.0883 - val_f1: 0.8006
Epoch 53/2000
10000/10000 [==============================] - 2s 167us/sample - loss: 1.1016 - f1: 0.7913 - val_loss: 1.1025 - val_f1: 0.7930
Epoch 54/2000

15000/15000 [==============================] - 2s 160us/sample - loss: 1.1186 - f1: 0.7724 - val_loss: 1.0366 - val_f1: 0.8072
Epoch 37/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 1.1124 - f1: 0.7698 - val_loss: 1.1784 - val_f1: 0.7707
Epoch 38/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 1.0679 - f1: 0.7848 - val_loss: 0.9798 - val_f1: 0.8163
Epoch 39/2000
15000/15000 [==============================] - 2s 160us/sample - loss: 1.0940 - f1: 0.7740 - val_loss: 1.0427 - val_f1: 0.7982
Epoch 40/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 1.0543 - f1: 0.7850 - val_loss: 1.1429 - val_f1: 0.7605
Epoch 41/2000
15000/15000 [==============================] - 2s 160us/sample - loss: 1.0282 - f1: 0.7896 - val_loss: 0.9487 - val_f1: 0.8264
Epoch 42/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 1.0237 - f1: 0.7876 - val_loss: 0.9761 - val_f1: 0.8152
Epoch 43/2000
15000/15000 [

Epoch 43/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 1.0241 - f1: 0.7953 - val_loss: 0.9744 - val_f1: 0.8064
Epoch 44/2000
15000/15000 [==============================] - 2s 160us/sample - loss: 0.9855 - f1: 0.8072 - val_loss: 0.9554 - val_f1: 0.8174
Epoch 45/2000
15000/15000 [==============================] - 2s 160us/sample - loss: 1.0025 - f1: 0.7979 - val_loss: 0.9913 - val_f1: 0.7978
Epoch 46/2000
15000/15000 [==============================] - 2s 159us/sample - loss: 0.9833 - f1: 0.8010 - val_loss: 0.9918 - val_f1: 0.8039
Epoch 47/2000
15000/15000 [==============================] - 7s 484us/sample - loss: 0.9991 - f1: 0.7979 - val_loss: 0.9451 - val_f1: 0.8175
Running through fold 2
Train on 15000 samples, validate on 3000 samples
Epoch 1/2000
15000/15000 [==============================] - 7s 451us/sample - loss: 31.0617 - f1: 0.2093 - val_loss: 20.0794 - val_f1: 0.3456
Epoch 2/2000
15000/15000 [==============================] - 2s 160us/sample - los

Epoch 3/2000
15000/15000 [==============================] - 2s 160us/sample - loss: 11.0597 - f1: 0.4047 - val_loss: 9.4190 - val_f1: 0.4673
Epoch 4/2000
15000/15000 [==============================] - 2s 161us/sample - loss: 8.4881 - f1: 0.4578 - val_loss: 7.3352 - val_f1: 0.5271
Epoch 5/2000
15000/15000 [==============================] - 2s 161us/sample - loss: 6.8370 - f1: 0.4922 - val_loss: 6.0195 - val_f1: 0.5613
Epoch 6/2000
15000/15000 [==============================] - 2s 163us/sample - loss: 5.6674 - f1: 0.5262 - val_loss: 5.0930 - val_f1: 0.5711
Epoch 7/2000
15000/15000 [==============================] - 2s 163us/sample - loss: 4.7820 - f1: 0.5553 - val_loss: 4.3642 - val_f1: 0.5917
Epoch 8/2000
15000/15000 [==============================] - 2s 163us/sample - loss: 4.1555 - f1: 0.5811 - val_loss: 3.7274 - val_f1: 0.6245
Epoch 9/2000
15000/15000 [==============================] - 2s 163us/sample - loss: 3.6714 - f1: 0.5911 - val_loss: 3.3306 - val_f1: 0.6643
Epoch 10/2000
15000